# Run AR6-WG3 fair but focus on saving out the two ocean layers

## import required packages

In [1]:
import fair
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import json
from tqdm import tqdm
import pickle
import matplotlib.pyplot as pl

## get SSP2-4.5 emissions pathways (original CMIP6 version)

In [2]:
ssp_df = pd.read_csv('../data_input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
years = np.arange(1750, 2021)

startyear = 1750
first_scenyear = 2015
last_scenyear = 2020
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

species = [
    '|CO2|MAGICC Fossil and Industrial',
    '|CO2|MAGICC AFOLU',
    '|CH4',
    '|N2O',
    '|Sulfur',
    '|CO',
    '|VOC',
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|CF4',
    '|C2F6',
    '|C6F14',
    '|HFC23',
    '|HFC32',
    '|HFC4310mee',
    '|HFC125',
    '|HFC134a',
    '|HFC143a',
    '|HFC227ea',
    '|HFC245fa',
    '|SF6',
    '|CFC11',
    '|CFC12',
    '|CFC113',
    '|CFC114',
    '|CFC115',
    '|CCl4',
    '|CH3CCl3',
    '|HCFC22',
    '|HCFC141b',
    '|HCFC142b',
    '|Halon1211',
    '|Halon1202',
    '|Halon1301',
    '|Halon2402',
    '|CH3Br',
    '|CH3Cl',
]

unit_convert = np.ones(40)
unit_convert[1] = 12/44/1000
unit_convert[2] = 12/44/1000
unit_convert[4] = 28/44/1000
unit_convert[5] = 32/64
unit_convert[8] = 14/46

emissions_out = np.ones((271, 40)) * np.nan
emissions_out[:,0] = years

years_future = [2015, 2020]
for i, specie in enumerate(species):
    emissions_out[:first_row,i+1] = ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable'].str.endswith(specie)),str(startyear):'2014']*unit_convert[i+1]
    f = interp1d(years_future, ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable'].str.endswith(specie)), '2015':'2020'].dropna(axis=1))
    emissions_out[first_row:(last_row+1), i+1] = f(
        np.arange(first_scenyear, last_scenyear+1)
    )*unit_convert[i+1]

## get FaIR v1.6.2 AR6 config

It's ok to run 1.6.4 with the 1.6.2 config, they are the same model but the slightly later version enables multigas restarts

In [3]:
with open('../data_input/fair-1.6.2/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

## run FaIR

In [6]:
def run_fair(args):
    c, f, t, _, _, _, _ = fair.forward.fair_scm(**args)
    return (
        c[:,0],
        np.sum(f, axis=1), 
        t-np.mean(t[100:151]),
    )

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions[:266]
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
        solar_truncated = updated_config[i]["F_solar"][:266]
        updated_config[i]['F_solar'] = solar_truncated
        volcanic_truncated = updated_config[i]["F_volcanic"][:266]
        updated_config[i]['F_volcanic'] = volcanic_truncated
        nat_truncated = updated_config[i]['natural'][:266, :]
        updated_config[i]['natural'] = nat_truncated
        
    shape = (266, len(updated_config))
    c_co2 = np.ones(shape) * np.nan
    t = np.ones(shape) * np.nan
    f_tot = np.ones(shape) * np.nan
    
    for i, cfg in tqdm(enumerate(updated_config), total=len(updated_config), position=0, leave=True):
        c_co2[:,i], f_tot[:,i], t[:,i] = run_fair(updated_config[i])


    return c_co2, f_tot, t

In [7]:
results_out = {}
(
    _,
    _,
    results_out['temperatures'],
) = fair_process(emissions_out)

100%|███████████████████████████████████████| 2237/2237 [03:26<00:00, 10.84it/s]


In [9]:
results_out['temperatures']

array([[-0.03155906, -0.02997186, -0.00632323, ..., -0.03067784,
        -0.05260862, -0.02570833],
       [-0.0095842 , -0.01187964,  0.01235815, ..., -0.0093414 ,
        -0.02956206, -0.00300636],
       [ 0.00678337, -0.00249307,  0.01783745, ...,  0.00341315,
        -0.0150688 ,  0.01201941],
       ...,
       [ 1.13082859,  1.03954602,  0.8588632 , ...,  1.20480302,
         1.18292661,  1.27450852],
       [ 1.15625442,  1.06142498,  0.88018399, ...,  1.23200088,
         1.21233706,  1.30814952],
       [ 1.18242527,  1.08027031,  0.89837056, ...,  1.25760808,
         1.2411609 ,  1.34113701]])

In [14]:
np.median(results_out['temperatures'][245:265,:].mean(axis=0))

0.8811159557777971

In [15]:
np.median(results_out['temperatures'][265,:].mean(axis=0))

1.1400066728830103